# Writing Unit Tests

Teacher: [Moshe Zadka](https://cobordism.com)

Start time: **15:00 US/Eastern**

## Acknowledgement of Country

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone


## Equipment check

### Jupyter

Run and connect to Jupyter.

If you installed it locally,
`http://localhost:8888`
is the default.

### ipytest

In [ ]:
import ipytest
ipytest.autoconfig()

There should not be any output from this step. If an error occured saying "module not found", make sure the virtual environment has `ipytest` installed.

### Writing and running tests

In [ ]:
%%run_pytest[clean]

import pytest

@pytest.mark.parametrize('value', [1, 2])
def test_something(value):
    assert value != value

### Self-check

Run it yourself.

Check to see the same thing happened!

## Assertions

### Example Test

In [ ]:
%%run_pytest[clean] -vv

def test_something():
    assert 1 == 1 + 1

### What are assertions

A test is a combination of two things:

* Running the "system under test"
* Checking the results

Assertions help check the result is correct.
Unless the only goal of the test is to check the SUT
ran without errors, you will need to check something.

Assertions in `pytest` use the Python `assert` statement.

The statement checks that its input is a truthy value,
and otherwise raises an `AssertionError`

In [ ]:
try:
    assert 1 == 0
except AssertionError as exc:
    print(repr(exc))

In [ ]:
try:
    assert 1 == 0, "math is still ok"
except AssertionError as exc:
    print(repr(exc))

In [ ]:
try:
    assert 1 == 1, "math is weird"
except AssertionError as exc:
    print(repr(exc))

### How Pytest handles assertions

In [ ]:
%%run_pytest[clean] -vv

import pytest

def test_math():
    assert 1 == 0

When tests are running in `pytest`,
it modifies the `assert` statement
so that it can give more informative errors.

It will give diffs, further details,
or explanations as appropriate.

### Reading assertion failures

The first part is the *running output*.

You will often see this in build log,
or on the console,
*while* the tests are running.

It shows you the status of each test.

```
============================= test session starts ==============================
platform linux -- Python 3.9.0, pytest-6.1.2, py-1.9.0, pluggy-0.13.1 -- /opt/carme/venvs/testing-in-python/bin/python
cachedir: .pytest_cache
rootdir: /opt/carme/src/testing-in-python/session-1
collecting ... collected 1 item

tmpz_pjyxpu.py::test_something FAILED                                    [100%]
```

The next part is the
*failure details section*:

This will have a subsection for each failing test.

It shows you details about the failure:

* Code snippet
* Values of relevant parts of the assertion
* (Sometimes) a diff

```
=================================== FAILURES ===================================
________________________________ test_something ________________________________

    def test_something():
>       assert 1 == 1 + 1
E       assert 1 == 2
E         +1
E         -2

<ipython-input-3-8b025c28988b>:2: AssertionError
```

The last part is the failure summary. It shows you how many tests succeeded and failed,
and a few details about each success/failure.

The previous example was so minimal, it is not obvious how the parts relate.
You can learn from a slightly more complicated example:

In [ ]:
%%run_pytest[clean] -vv

def test_error():
    assert 0 == 1
    
def test_succeed():
    assert 1 == 1
    
def test_fail():
    assert 1/0 == 0

Here are a few things to notice:

* A succeeding test is not mentioned by name in the summary, but it is counted
* A failure that is *not* caused by an assertion is a lot less obvious.

### Equality

Here is an example: you have a function that implements addition.
Unfortunately, it has a bug -- it returns the expected result with a small difference.

In [ ]:
%%run_pytest[clean] -vv

def add(a, b):
    result = 0.1 # This is a mistake -- should be 0
    result += a
    result += b
    return result

def test_add():
    assert add(3, 4) == 7

This example contains few lines of code, but still has some subtleties to unpack:

* The test was comparing `add()` to the *expected result* because, presumably, this is what the function was documented to do.

* The test did not care about the *implementation* of `add()`. 

Another way to implement `add()` (incorrectly) is here:

In [ ]:
%%run_pytest[clean] -vv

def add(a, b):
    return (
        a +
        b + 
        0.1 # This is a mistake -- should be 0
    )

def test_add():
    assert add(3, 4) == 7

The test failed in precisely the same way. The test only checks `add()` against the documented guarantees.

Because in this case, the documented guarantees are strict and possible to plan for,
this test works well with an equality assertion.

With lists
there are three potential things that can happen.

1. The list on the right might be a proper prefix of the list on the left.

In [ ]:
%%run_pytest[clean] -vv

def test_missing():
    assert [1] == []

2. The list on the left might be a proper prefix of the list on the right.

In [ ]:
%%run_pytest[clean] -vv

def test_extra():
    assert [1] == [1, 2]

3. At least in one index, the element at that index in both lists is different:

In [ ]:
%%run_pytest[clean] -vv

def test_different():
    assert [1, 2, 3] == [1, 4, 3]

These failures are *different*.
Learning how to read the failures,
and understanding what's wrong is important.
This is not just important when troubleshooting a failing test.
It is also important when writing tests:
remember, every assertion has to pay rent by being simulated
with a bug in the system under test.

It is common to compare strings:
from expected output to running external commands,
many things are strings that have strict equality guarantees.

In [ ]:
%%run_pytest[clean] -vv

def test_string():
    assert "hello\nworld" == "goodbye\nworld"

Depending on the length of the string, sometimes an inside-the-line diff will be triggered:

In [ ]:
%%run_pytest[clean] -vv

def test_string():
    assert "saying " * 10 + "hello world" + " said" * 10 == "saying " * 10 + "goodbye world" + " said" * 10

`pytest` will try to give useful diffs with most containers:

In [ ]:
%%run_pytest[clean] -vv

def test_set():
    assert set([1,2]) == set([1,3])

For sets, it will check for spurious elements on both sides.

## Tip -- Inserting Exceptions

Inserting exceptions into tests is a surprisingly good way to debug test failures.

* You can make sure the test fails: test fail with exceptions!
* The exception -> test failure output is the most reliable output path.

In [ ]:
%%run_pytest[clean] -vv

def subtle_manipulation(things):
    trim = 1 # It should be 2
    return things[trim:-trim]

def test_subtle_manipulation():
    assert subtle_manipulation("--HELLO--") == "HELLO"

def erroring_subtle_manipulation(things):
    trim = 1
    ret_value = things[trim:-trim]
    raise ValueError(things, ret_value, trim)
    return ret

def test_subtle_manipulation_2():
    assert erroring_subtle_manipulation("--HELLO--") == "HELLO"


### Other assertions

Usually inequality is asserted as a "helper" assertion.
For example, if a function is defined to produce a "different" example for something,
it might be reasonable to also verify that these examples are all indeed different.

Pytest will still print the values,
but in this case, there is no diff:

In [ ]:
%%run_pytest[clean] -vv

def test_not_equal():
    x = 1
    y = x / 1
    assert x != y

Inequality is not just for numbers:
there are other objects that might need to be compared for inequality.
Again, notice what pytest does and does not output.

In [ ]:
%%run_pytest[clean] -vv

def test_not_equal_lists():
    x = [1, 2, 3]
    assert x != x[:]

A stronger assertion is *order*.
Both the `<` and `<=` operators, as well as their inverses, can be useful.

In [ ]:
%%run_pytest[clean] -vv

def test_greater():
    x = 1
    assert x < x

Be mindful of which ordering operator you want:
always think about the semantics and whether `<` or `<=` is appropriate.

In [ ]:
%%run_pytest[clean] -vv

def test_greater_or_equal():
    x = 1
    assert x + 1 <= x

Sets can also be compared.
Note that there is no diff.
Compare two tests:

In [ ]:
%%run_pytest[clean] -vv

SMALLER = {1, 2, 3}
BIGGER = {1, 2}

def test_set_comparison():
    assert SMALLER <= BIGGER
    
def test_set_comparison_with_equality():
    assert SMALLER & BIGGER == SMALLER

The `in` and `not in` operators are sometimes useful to assert about items and containers.

In [ ]:
%%run_pytest[clean] -vv

def test_in():
    assert 5 in range(3)
    
def test_not_in():
    assert 1 not in set([1])

In some cases, you want to verify the actual identity of objects.
Note that the behavior of `is` on built-in constants is awkwardly defined.

In [ ]:
%%run_pytest[clean] -vv

def test_identical():
    assert [1, 2, 3] is [1, 2, 3]
    
def test_not_identical():
    x = "hello"
    assert x is not x

All comparison operators in Python (`==` and the others)
*chain*.
Pytest special-cases comparison chains.

In [ ]:
%%run_pytest[clean] -vv

def test_equality_chain():
    x = [1, 2, 3]
    assert [1, 2, 3] == x == [1, 2, 3, 4]

Sometimes there will be a direct boolean you want to assert.
In that case, it is useful to put the place this boolean came from
in the test assertion.

In [ ]:
%%run_pytest[clean] -vv

def always_false(something):
    return False

def test_false():
    x = [1, 2, 3]
    x.append(4)
    x.append(4)
    assert always_false(x)

### Exercise

Video will pause for 10m

In [ ]:
%%run_pytest[clean] -vv

def safe_remove(a, b):
    pass # fix this line

def test_safe_remove_no():
    things = {1: "yes", 2: "no"}
    safe_remove(things, 3)
    assert 1 in things

def test_safe_remove_yes():
    things = {1: "yes", 2: "no"}
    safe_remove(things, 2)
    assert 2 not in things

In [ ]:
%%run_pytest[clean] -vv

def get_min_max(a, b):
    return a, b # fix this line

def test_min_max_high():
    a, b = get_min_max(2, 1)
    assert set([a, b]) == set([1, 2])
    assert a < b

def test_min_max_low():
    a, b = get_min_max(1, 2)
    assert set([a, b]) == set([1, 2])
    assert a < b

### Solving exercise

In [ ]:
%%run_pytest[clean] -vv

def safe_remove(a, b):
    a.pop(b, None)

def test_safe_remove_no():
    things = {1: "yes", 2: "no"}
    safe_remove(things, 3)
    assert 1 in things

def test_safe_remove_yes():
    things = {1: "yes", 2: "no"}
    safe_remove(things, 2)
    assert 2 not in things

In [ ]:
%%run_pytest[clean] -vv

def get_min_max(a, b):
    return min([a, b]), max([a, b])

def test_min_max_high():
    a, b = get_min_max(2, 1)
    assert set([a, b]) == set([1, 2])
    assert a < b

def test_min_max_low():
    a, b = get_min_max(1, 2)
    assert set([a, b]) == set([1, 2])
    assert a < b

### Summary

* Tests: run + verify
* Assertions verify
* Pytest rewrites `assert`

## Break

Video will pause for 10m

## Mock basics

### What are mocks

Mocks are found in the module `unittest.mock` in the standard library.

The most common class is `MagicMock()`, and will be the main one this class covers.

In order to see mocks in action, you can intentionally fail some tests.

Testing a function that raises an exception is a good way to see how mocks work.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def raise_value(x):
    raise ValueError(x)

def test_value():
    raise_value(mock.MagicMock())

### Default mock properties

Mocks have every attribute. It is also a mock.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def raise_some_name(x):
    raise ValueError(x.some_name)

def test_name():
    raise_some_name(mock.MagicMock())

A mock's attribtues are *consistent*.
They stay the same, so retrieving the same attribute again gives identical objects.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_consistent():
    obj = mock.MagicMock()
    assert obj.some_name is not obj.some_name

### Calling mocks

Mocks can be called. They return a mock.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def raise_call(x):
    raise ValueError(x())

def test_examine():
    raise_call(mock.MagicMock())

When calling a mock again. it will again return the same value.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_consistent_call():
    obj = mock.MagicMock()
    assert obj() is not obj()

Because an attribute returns a mock, and mocks can be called,
mocks also have all the methods.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def raise_deep(x):
    val = x.some_method()
    raise ValueError(val.some_attribute)
    
def test_deep():
    raise_deep(mock.MagicMock())

### Mock magic methods

The `Magic` in `MagicMock` is because it also has the so-called "magic methods".

Those are the methods that allow objects to overload operations like addition.

This means mocks can also be added.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def raise_add_1(x):
    val = x + 1
    raise ValueError(val)
    
def test_add_1():
    raise_add_1(mock.MagicMock())

Because calling methods on a mock returns the same value, it does not matter what we add.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock
    
def test_add_different():
    x = mock.MagicMock()
    assert x + 1 != x + 5

Iterating over values is possible, but mocks don't yield any elements.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def iterate_over(x):
    for el in x:
        raise ValueError(el)
    raise ValueError("no elements", x)
    
def test_iterate():
    iterate_over(mock.MagicMock())

Regardless whether you use the `[]` operator with indices, slices, or step-slices, mocks will return the same thing.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def raise_index(x):
    raise ValueError(x[5])
    
def raise_slice(x):
    raise ValueError(x[5:7])

def raise_step_slice(x):
    raise ValueError(x[::-1])
    
def test_index():
    raise_index(mock.MagicMock())

def test_slice():
    raise_slice(mock.MagicMock())
    
def test_step_slice():
    raise_step_slice(mock.MagicMock())

Mocks can be *named*.

Naming mocks is a good practice. This makes many errors easier to disagnose from the exception or assertion message.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def copy_stuff(source, target):
    write_to(target, source, 10)
    
def write_to(source, target, length):
    stuff = source.read()
    target.write(stuff)
    raise ValueError(source, target, stuff)

def test_opaque():
    source = mock.MagicMock()
    target = mock.MagicMock()
    copy_stuff(source, target)
    
def test_clear():
    source = mock.MagicMock(name="source")
    target = mock.MagicMock(name="target")
    copy_stuff(source, target)


Compare 

```
ValueError: (<MagicMock id='140019688468192'>, <MagicMock id='140019688435776'>, <MagicMock name='mock.read()' id='140019688426272'>)
```

with 

```
ValueError: (<MagicMock name='target' id='140019688082976'>, <MagicMock name='source' id='140019895734528'>, <MagicMock name='target.read()' id='140019895731680'>)
```

With `target.read()`, the bug is much easier to spot!

### Setting properties and deep properties

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_attribute():
    x = mock.MagicMock(name="thing")
    x.some_attribute = 5
    assert x.some_attribute != 5

You can also set the attribute in the constructor.
This is usually better, because there is no step
where the mock object is not correct:

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_attribute_constructor():
    x = mock.MagicMock(name="thing", some_attribute=5)
    assert x.some_attribute != 5

Because by default every attribute on a mock is a mock itself,
you can set "deep attributes" that violate the "law" of Demeter.

In this context, it's fine -- although some argue that the test
is not a "unit test" if this is needed.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_deep_attribute():
    x = mock.MagicMock(name="thing")
    x.some_attribute.value = 5
    assert x.some_attribute.value != 5

You can also set several attributes,
deep or otherwise,
on a mock at the same time using
`configure_mock`.
This is no different than using the constructor.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_config_mock():
    x = mock.MagicMock(name="thing")
    x.configure_mock(**{
        "some_attribute.value": 5,
        "gauge": 7,
    })
    assert x.some_attribute.value != 5 or x.gauge != 7

### Exercise

Video will pause for 10 minutes

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def add_1(x):
    return x.value + 1

def test_deep_attribute():
    x = mock.MagicMock(name="thing")
    pass # Change only this line
    assert add_1(x.some_attribute) == 2

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock
def add_1(x):
    return x.value + 1

   
def test_config_mock():
    x = mock.MagicMock(name="thing")
    x.configure_mock(**{
        # Change only this line
        "gauge": 7,
    })
    assert add_1(x.some_attribute) == 2

### Solving exercise

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def add_1(x):
    return x.value + 1

def test_deep_attribute():
    x = mock.MagicMock(name="thing")
    x.some_attribute.value = 1 # pass # Change only this line
    assert add_1(x.some_attribute) == 2

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock
def add_1(x):
    return x.value + 1

   
def test_config_mock():
    x = mock.MagicMock(name="thing")
    x.configure_mock(**{
        "some_attribute.value": 1, # Change only this line
        "gauge": 7,
    })
    assert add_1(x.some_attribute) == 2

### Summary

* Name mocks
* Set properties and "deep" properties

## Break

Video will pause for 10 minutes

## Advanced Mocks

### Mock return value

Python's duck-typing means that often what we want to do with mock objects is *call them* (or *call methods on them*) and get specific return values. 

A mock object returns whatever value is in its `return_value` attribute.

Like any attribute, a mock object will have that attribute,
and it will be a mock by default.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_use_return_value():
    obj = mock.MagicMock(name="obj")
    obj.return_value.some_attribute = 5
    assert obj().some_attribute != 5

As in other cases of "deep attributes", you can set it in the constructor with the right "keyword arguments".

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_use_return_value_constructor():
    obj = mock.MagicMock(name="obj", **{"return_value.some_attribute": 5})
    assert obj().some_attribute != 5

You can set the `return_value` property itself, which will return a regular value.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_set_return_value():
    obj = mock.MagicMock(name="obj")
    obj.return_value = 5
    assert obj() != 5

You can set it using the constructor too. Here, a regular keyword argument will work.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_set_return_value_constructor():
    obj = mock.MagicMock(name="obj", return_value=5)
    assert obj() != 5

The most common occurence is wanting to set the return value of a *method*.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_set_rmethod_return_value():
    obj = mock.MagicMock(name="obj")
    obj.method.return_value = 5
    assert obj.method() != 5

Setting the return value of a method is a "deep attribute", so you will need to use the special syntax
to pass it in the constructor.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_set_rmethod_return_value_constructor():
    obj = mock.MagicMock(name="obj", **{"method.return_value": 5})
    assert obj.method() != 5

Putting all of these ideas together, in order to set an *attribute* on the return value of a *method*,
you will have to use the special syntax, and have *two* dots in the name.

In [ ]:
%%run_pytest[clean] -vv

from unittest import mock

def test_set_rmethod_deep_return_value_constructor():
    obj = mock.MagicMock(name="obj", **{"method.return_value.some_attribute": 5})
    assert obj.method().some_attribute != 5

### Mock side effect -- iterator


One of the things that can be assigned to `side_effect` is
an *iterable*, such as a sequence or a generator.

This is a powerful feature -- it allows controlling each call's return value,
with little code.

In [ ]:
%%run_pytest[clean]

from unittest import mock

def test_values():
    different_things = mock.MagicMock()
    different_things.side_effect = [1, 2, 3]
    assert different_things() == 1
    assert different_things() == 2
    assert different_things() == 4


A more realistic example is when simulating file input.
In this case, we want to be able to control what `readline` returns
each time to pretend it is file input.

In [ ]:
%%run_pytest[clean]

from unittest import mock

def parse_three_lines(fpin):
    line = fpin.readline()
    name, value = line.split()
    modifier = fpin.readline().strip()
    extra = fpin.readline().strip()
    return {name: f"{value}/{modifier}+{extra}"}

from io import TextIOBase
    
def test_parser():
    filelike = mock.MagicMock(spec=TextIOBase)
    filelike.readline.side_effect = [
        "thing important\n",
        "a-little\n",
        "to-some-people\n"
    ]
    value = parse_three_lines(filelike)
    assert value == dict(thing="important/a-little+to-most-people")


### Mock side effect -- function

Some code needs to have the mock objects do some non-trivial computation.
For example, when testing code for a network client.

Trying to test such code without performing any computation is difficult. The tests tend to be too *insensitive* or too *flakey*. An insensitive test is one that does not fail in the presence of bugs. A flakey test is one that sometimes fails, even when the code is  correct. Here, our code is incorrect. The insensitive test does not catch it, while the flakey test would fail even if it was fixed!

In [ ]:
%%run_pytest[clean]
import socket
import random

def yolo_reader(sock):
    sock.settimeout(5)
    sock.connect(("some.host", 8451))
    fpin = sock.makefile()
    order = [0, 1]
    random.shuffle(order)
    while order:
        if order.pop() == 0:
            sock.sendall(b"GET KEY\n")
            key = fpin.readline().strip()
        else:
            sock.sendall(b"GET VALUE\n")
            value = fpin.readline().strip()
    return {value: key} ## Woops bug, should be {key: value}
    
from io import TextIOBase
from unittest import mock
import pytest

def test_insensitive_test():
    sock = mock.MagicMock(spec=socket.socket)
    sock.makefile.return_value.readline.return_value = "interesting\n"
    assert yolo_reader(sock) == {"interesting": "interesting"}
    
@pytest.mark.parametrize("does_nothing", [1, 2, 3, 4, 5])
def test_flakey_test(does_nothing):
    sock = mock.MagicMock(spec=socket.socket)
    sock.makefile.return_value.readline.side_effect = ["key\n", "value\n"]
    assert yolo_reader(sock) == {"key": "value"}

The final option of getting results from a mock object is to assign a *callable object* to `side_effect`. This calls `side_effect` to simply call it. Why not just assign a callable object directly to the attribute? Have patience, we'll get to that in the next part!

In this example, our callable object (just a function) will assign a `return_value` to the attribute of another object. This is not that uncommon. We are simulating the environment, and in a real environment, poking one thing often has an effect on other things.

In [ ]:
%%run_pytest[clean]
import socket
import random

def yolo_reader(sock):
    sock.settimeout(5)
    sock.connect(("some.host", 8451))
    fpin = sock.makefile()
    order = [0, 1]
    random.shuffle(order)
    while order:
        if order.pop() == 0:
            sock.sendall(b"GET KEY\n")
            key = fpin.readline().strip()
        else:
            sock.sendall(b"GET VALUE\n")
            value = fpin.readline().strip()
    return {value: key} ## Woops bug, should be {key: value}
    
from io import TextIOBase
from unittest import mock

def test_yolo_well():
    sock = mock.MagicMock(spec=socket.socket)
    def sendall(data):
        cmd, name = data.decode("ascii").split()
        if name == "KEY":
            sock.makefile.return_value.readline.return_value = "key\n"
        elif name == "VALUE":
            sock.makefile.return_value.readline.return_value = "value\n"
        else:
            raise ValueError("got bad command", name)
    sock.sendall.side_effect = sendall
    assert yolo_reader(sock) == {"key": "value"}

### Mock call args and call args list

#### Call arguments

In the following example, we want to make sure the code calls the method with the *correct* arguments.
When automating data center manipulations, it is important to get things right.
As they say, "To err is human, but to destroy an entire data center requires a robot with a bug."

We want to make sure our Paramiko-based automation will correctly get the sizes of files, even when the file names have spaces in them.

In [ ]:
%%run_pytest[clean]

def get_remote_file_size(client, fname):
    client.connect('ssh.example.com')
    stdin, stdout, stderr = client.exec_command(f"ls -l {fname}")
    stdin.close()
    results = stdout.read()
    errors = stderr.read()
    stdout.close()
    stderr.close()
    if errors != '':
        raise ValueError("problem with command", errors)
    return int(results.split()[4])

import pytest
from unittest import mock
import shlex

@pytest.mark.parametrize("fname", ["readme.txt", "a file"])
def test_file_size(fname):
    client = mock.MagicMock()
    client.exec_command.return_value = [mock.MagicMock(name=str(i)) for i in range(3)]
    client.exec_command.return_value[1].read.return_value = f"""\
    -rw-rw-r--  1 user user    123 Jul 18 20:25 {fname}
    """
    client.exec_command.return_value[2].read.return_value = ""
    result = get_remote_file_size(client, fname)
    assert result == 123
    [args], kwargs = client.exec_command.call_args
    assert shlex.split(args) == ["ls", "-l", fname]

#### List of call args

Sometimes, this is not enough. Some code calls functions repeatedly, and we need to test that *all* calls are correct.
The most sophisticated X-Ray we have is `.call_args_list` which gives the entire history of what happened to the callable.

For this example, we will pretend that the (*real*) remote calculator API only allows multiplying two numbers. In order to *cube* the number, calculate `x**3`, we need two calls to the service. For superstitious reasons, we want to always put the bigger number first: maybe someone told us that it is faster this way.

In [ ]:
%%run_pytest[clean]

import httpx
from unittest import mock

def calculate_cube(client, base):
    square = int(client.get(f"https://api.mathjs.org/v4/?expr={base}*{base}").text) # x*x
    return int(client.get(f"https://api.mathjs.org/v4/?expr={base}*{square}").text) # x*x*x

def test_calculate_cube():
    client = mock.MagicMock(spec=httpx.Client)
    client.get.side_effect = [mock.MagicMock(text=str(x)) for x in [25, 125]]
    assert calculate_cube(client, 5) == 125
    assert client.get.call_count == 2
    squaring, cubing = client.get.call_args_list
    args, kwargs = squaring
    assert kwargs == {}
    assert args == tuple(["https://api.mathjs.org/v4/?expr=5*5"])
    args, kwargs = cubing
    assert kwargs == {}
    ## Make sure bigger number comes first!
    assert args == tuple(["https://api.mathjs.org/v4/?expr=25*5"])

### Exercise

Video will pause for 10 minutes

In [ ]:
%%run_pytest[clean]

import httpx
from unittest import mock
import pytest
import random

def calculate_fifth_power(client, base):
    square = int(client.get(f"https://api.mathjs.org/v4/?expr={base}*{base}").text)
    hypercube = int(client.get(f"https://api.mathjs.org/v4/?expr={square}*{square}").text)
    # random order
    args = [hypercube, base]
    random.shuffle(args)
    result = int(client.get(f"https://api.mathjs.org/v4/?expr={args[0]}*{args[1]}").text)
    return result

@pytest.mark.parametrize("does_nothing", [1, 2, 3, 4, 5])
def test_calculate_cube(does_nothing):
    client = mock.MagicMock(spec=httpx.Client)
    client.get.side_effect = [mock.MagicMock(text=str(x)) for x in [25, 625, 3125]]
    assert calculate_fifth_power(client, 5) == 3125
    assert client.get.call_count == 3
    squaring, hypercubing, final = client.get.call_args_list
    args, kwargs = squaring
    assert kwargs == {}
    assert args == tuple(["https://api.mathjs.org/v4/?expr=5*5"])
    args, kwargs = hypercubing
    assert kwargs == {}
    args, kwargs = final
    assert kwargs == {}
    [url] = args
    constant, expr = url.split("=", 1)
    assert constant == "https://api.mathjs.org/v4/?expr"
    assert expr == "625*5" # Change only this line

### Solving Exercise

In [ ]:
%%run_pytest[clean]

import httpx
from unittest import mock
import pytest
import random

def calculate_fifth_power(client, base):
    square = int(client.get(f"https://api.mathjs.org/v4/?expr={base}*{base}").text)
    hypercube = int(client.get(f"https://api.mathjs.org/v4/?expr={square}*{square}").text)
    # random order
    args = [hypercube, base]
    random.shuffle(args)
    result = int(client.get(f"https://api.mathjs.org/v4/?expr={args[0]}*{args[1]}").text)
    return result

@pytest.mark.parametrize("does_nothing", [1, 2, 3, 4, 5])
def test_calculate_cube(does_nothing):
    client = mock.MagicMock(spec=httpx.Client)
    client.get.side_effect = [mock.MagicMock(text=str(x)) for x in [25, 625, 3125]]
    assert calculate_fifth_power(client, 5) == 3125
    assert client.get.call_count == 3
    squaring, hypercubing, final = client.get.call_args_list
    args, kwargs = squaring
    assert kwargs == {}
    assert args == tuple(["https://api.mathjs.org/v4/?expr=5*5"])
    args, kwargs = hypercubing
    assert kwargs == {}
    args, kwargs = final
    assert kwargs == {}
    [url] = args
    constant, expr = url.split("=", 1)
    assert constant == "https://api.mathjs.org/v4/?expr"
    assert set(expr.split("*")) == {"625", "5"} # Change only this line

## Final thoughts and Q&A

### Putting it all together

* Assertions let you verify conditions
* Mocks let you avoid interacting with "real life"
* You can set mock attributes, return values, or even behavior
* Mocks record arguments they were called with

### Testable code

* This shows how to test -- but writing testable code is important!
* The power of mocks shines when passing arguments to functions or initializers -- do it more.
* Patching is a tool of last resort.

### Q&A